In [5]:
import microfpga.controller as cl
import microfpga.signals as sig
from microfpga.signals import LaserTriggerMode

import time

# 0 - Connect to the FPGA

It is important to close the USB port after instantiating it. One convenient way to do it is with a `with` statement.

In [2]:
# with cl.MicroFPGA(know_device=XXX) as mufpga: <- replace XXX with the correct port if there are multiple ports
with cl.MicroFPGA() as mufpga:

    # check if successful
    if mufpga.is_connected():

        # print id
        print(f'Connected to {mufpga.get_id()}')

    else:
        print('Failed to connect')

print('Disconnected')

Failed to connect
Disconnected


/home/joran.deschamps/git/MicroFPGA/MicroFPGA-py/microfpga/regint.py:74: UserWarning: Cannot choose between detected devices ['/dev/ttyUSB1', '/dev/ttyUSB0'] (known_device = None). Choose a device from the list and pass it as known_device parameter to the controller. If there is no detected device in the list, check the physical device connection.
  warnings.warn(f'Cannot choose between detected devices {devices} (known_device '


If you prefer to handle the connection and disconnection yourself, you can simply instantiate a MicroFPGA object.

In [9]:
# Instantiate MicroFPGA
mufpga = cl.MicroFPGA()
print(mufpga.is_connected())

# Check board ID
if mufpga.is_connected():
    print(f'Connected to {mufpga.get_id()}')

# Disconnect
mufpga.disconnect()
print(mufpga.is_connected())

False


AttributeError: 'MicroFPGA' object has no attribute '_id'

# 1 - Using a single laser

In [4]:
# By default use_camera = True, here we use passive synchronization
with cl.MicroFPGA(n_laser=1, use_camera=False) as mufpga:

    # check if successful
    if mufpga.is_connected():

        # print id
        print(f'Connected to {mufpga.get_id()}')

        # check current state of the first laser
        # it prints [mode, duration, sequence]
        laser_id = 0  # signals id are 0-indexed
        print(f'Current Laser {laser_id} state: {mufpga.get_laser_state(laser_id)}')

        # define a new laser state
        laser = {
            'channel': laser_id,
            'mode': LaserTriggerMode.MODE_RISING,  # pulsing on rising edge of camera input
            'duration': 1_000,  # pulse length
            'sequence': sig.format_sequence('1010101010101010')  # sequence of on/off frames
        }

        # apply the parameters at once
        mufpga.set_laser_state(**laser)

        # read current laser state
        print(f'Current Laser 0 state: {mufpga.get_laser_state(0)}')
        assert [laser['mode'].value,
                laser['duration'],
                laser['sequence']] == mufpga.get_laser_state(laser['channel'])

        # we can change again the parameters
        laser['mode'] = LaserTriggerMode.MODE_FALLING
        laser['duration'] = 2000
        laser['sequence'] = sig.format_sequence('0011001100110011')
        mufpga.set_laser_state(**laser)

        # read the new state
        print(f'Current Laser 0 state: {mufpga.get_laser_state(0)}')

    else:
        print('Failed to connect')

print('Disconnected')


Failed to connect
Disconnected


# 2 - Use multiple lasers

In [ ]:
with cl.MicroFPGA(n_laser=3, use_camera=False) as mufpga:

    # check if successful
    if mufpga.is_connected():

        # print id
        print(f'Connected to {mufpga.get_id()}')

        # check current state of the first laser
        # it prints [mode, duration, sequence]
        print(f'Current Laser 0 state: {mufpga.get_laser_state(0)}')
        print(f'Current Laser 1 state: {mufpga.get_laser_state(1)}')
        print(f'Current Laser 2 state: {mufpga.get_laser_state(2)}')

        # set lasers state
        laser0 = {
            'channel': 0,
            'mode': LaserTriggerMode.MODE_RISING,
            'duration': 2000,
            'sequence': sig.format_sequence('1010101010101010')
        }
        laser1 = {
            'channel': 1,
            'mode': LaserTriggerMode.MODE_FALLING,
            'duration': 2000,
            'sequence': sig.MAX_SEQUENCE  # sequence = 1111111111111111
        }
        laser2 = {
            'channel': 2,
            'mode': LaserTriggerMode.MODE_FOLLOW,
            'duration': 0,  # duration has no impact in FOLLOW mode
            'sequence': sig.format_sequence('1100110011001100')
        }
        mufpga.set_laser_state(**laser0)
        mufpga.set_laser_state(**laser1)
        mufpga.set_laser_state(**laser2)

        # read lasers state
        print(f'Current Laser 0 state: {mufpga.get_laser_state(0)}')
        print(f'Current Laser 1 state: {mufpga.get_laser_state(1)}')
        print(f'Current Laser 2 state: {mufpga.get_laser_state(2)}')

        assert [laser2['mode'].value,
                laser2['duration'],
                laser2['sequence']] == mufpga.get_laser_state(laser2['channel'])

    else:
        print('Failed to connect')

print('Disconnected')

# 3 - Active synchronization

In [ ]:
# use_camera=True is the default value and can be omitted
with cl.MicroFPGA(n_laser=1) as mufpga:
    # check if successful
    if mufpga.is_connected():

        # print id
        print(f'Connected to {mufpga.get_id()}')

        # we are in camera-laser sync mode by default
        print(f'Active trigger synchronisation: {mufpga.is_active_sync()}')
        assert mufpga.is_active_sync()

        # then we need to set the camera state
        # we can do it in milliseconds, keeping in mind the following bounds:
        # max(pulse) = 0 to 1048,575 ms (in steps of 1 us)
        # max(delay) = 0 to 65.535 ms
        # max(exposure) = 0 to 1048,575 ms
        # max(readout) = 0 to 65.535 ms
        camera = {
            'pulse': 1,  # ms
            'delay': 0.5,  # delay of 500 us between camera pulse and start of the exposure
            'exposure': 19.5,
            'readout': 1,
        }
        mufpga.set_camera_state_ms(**camera)  # set the values in ms

        # print state
        print(mufpga.get_camera_state_ms())

        # define one laser pulsing on rising edge of the
        # camera trigger with pulse length 2000 us.
        laser0 = {
            'channel': 0,
            'mode': LaserTriggerMode.MODE_RISING,
            'duration': 2000,  # in us
            'sequence': sig.MAX_SEQUENCE
        }
        mufpga.set_laser_state(**laser0)

        # we also need to start the camera
        mufpga.start_camera()
        print('Camera running')

        # now the FPGA generates both camera and laser trigger for 2 s
        time.sleep(2)  # in s

        assert mufpga.is_camera_running()

        # stop, the trigger signals are off
        mufpga.stop_camera()
        print('Camera stopped')

    else:
        print('Failed to connected')

print('Disconnected')

# 4 - Switch between active and passive synchronization

In [6]:
with cl.MicroFPGA(n_laser=1) as mufpga:
    # check if successful
    if mufpga.is_connected():

        # print id
        print(f'Connected to {mufpga.get_id()}')

        # we are in camera-laser sync mode by default
        print(f'Active trigger synchronisation: {mufpga.is_active_sync()}')
        assert mufpga.is_active_sync()

        # then we set the camera state
        camera = {
            'pulse': 1,  # ms
            'delay': 0.5,  # delay of 500 us between camera pulse and start of the exposure
            'exposure': 19.5,
            'readout': 21
        }
        mufpga.set_camera_state_ms(**camera)  # set the values in ms

        # print state
        print(mufpga.get_camera_state_ms())

        # define one laser pulsing on rising edge of the
        # camera trigger with pulse length 2000 us.
        laser0 = {
            'channel': 0,
            'mode': LaserTriggerMode.MODE_RISING,
            'duration': 2000,  # in us
            'sequence': sig.MAX_SEQUENCE
        }
        mufpga.set_laser_state(**laser0)

        # we also need to start the camera
        mufpga.start_camera()
        print('Camera running')

        # now the FPGA generates both camera and laser trigger for 2 s
        time.sleep(2)  # in s
        assert mufpga.is_camera_running()

        # stop, the trigger signals are off
        mufpga.stop_camera()
        print('Camera stopped')

        # switch to passive sync mode
        mufpga.set_passive_sync()
        print(f'Active trigger synchronisation: {mufpga.is_active_sync()}')
        assert not mufpga.is_active_sync()

        # and switch back
        mufpga.set_active_sync()
        print(f'Active trigger synchronisation: {mufpga.is_active_sync()}')
        assert mufpga.is_active_sync()

    else:
        print('Failed to connected')

print('Disconnected')

Failed to connected
Disconnected


# 5 - Analog read-input

In [7]:
def current_milli_time():
    return time.time() * 1000


def run_measurement(controller, channel, n):
    t = []
    r = []

    start = current_milli_time()
    for _ in range(n):
        r.append(controller.get_analog_state(channel))
        t.append(current_milli_time() - start)

    return t, r


with cl.MicroFPGA(n_ai=1) as mufpga:
    # check if successful
    if mufpga.is_connected():
        print('Connected to ' + mufpga.get_id())

        t_ms, r_au = run_measurement(mufpga, channel=0, n=100)

        # result is returned in arbitrary unit, convert to volts
        v = [r / sig.MAX_AI for r in r_au]

        print(f'Results: {v}')

    else:
        print('Failed to connect')

print('Disconnected')

Failed to connect
Disconnected


# 6 - Other signals

In [8]:
with cl.MicroFPGA(n_ttl=2,
                  n_servo=2,
                  n_pwm=3,
                  use_camera=False) as mufpga:

    # check if successful
    if mufpga.is_connected():

        # print id
        print(f'Connected to {mufpga.get_id()}')

        # TTL signals have only two states: on (True) or off (False)
        ttl_id = 1 # let's use the second channel
        ttl_state = mufpga.get_ttl_state(ttl_id)
        print(f'Current TTL {ttl_id} state: {ttl_state}')

        # set it to the other state
        mufpga.set_ttl_state(ttl_id, (not ttl_state))
        print(f'Current TTL {ttl_id} state: {mufpga.get_ttl_state(ttl_id)}')

        # PWM signals go from 0 (0%) to 255 (100%), they can be used together
        # with a low-pass filter to create an analog output signal, or simply
        # directly used with certain devices (e.g. LED)
        pwm_id = 2  # let's use the third signal
        pwm_state = mufpga.get_pwm_state(pwm_id)
        print(f'Current PWM {pwm_id} state: {pwm_state}')

        # let's change the value
        pwm_state = (pwm_state+120) % 255  # make sure the value is not > 255
        mufpga.set_pwm_state(pwm_id, pwm_state)
        print(f'Current PWM {pwm_id} state: {mufpga.get_pwm_state(pwm_id)}')

        # Finally, servo signals are used to control servomotors and their value
        # is between 0 and 65535
        servo_id = 1  # let's use the second channel
        servo_state = mufpga.get_servo_state(servo_id)
        print(f'Current Servo {servo_id} state: {servo_state}')

        # let's change the value
        servo_state = (servo_state+25000) % 65535  # make sure the value is not > 65535
        mufpga.set_servo_state(servo_id, servo_state)
        print(f'Current Servo {servo_id} state: {mufpga.get_servo_state(servo_id)}')

    else:
        print('Failed to connect')

print('Disconnected')

Failed to connect
Disconnected
